# Toxic Comments Classification - NLP & ML Project

With the advent of social networks, blogs and online chats the human psychology is changing: **likes, comments and tags are having a real and concrete role in our lives**, going from a "virtual" world to reality. The impact of **toxic comments affects the engagement of users** in a meaningful conversation. The threat of abuse and harassment online has a consequence that many people stop expressing themselves and give up (i.e. comments that are rude, disrespectful are likely to make someone leave a discussion). 

This means that **platforms and users loose the ability to be involved in constructive conversations.**  Moreover, the severity and viciousness of these comments has evolved into something much more sinister such as the 2021 US capitol riots and several cases of suicide, even among very young people. Identify toxic comments can **lead communities to facilitate and improve online conversations and exchange of opinions**.

## Abstract
After importing the data, that is raw comments, we proceed with an **initial cleaning of the text**, with the aim of removing textual elements that are not to be considered during the subsequent analysis (mentions, hashtags, repeated spaces, ...). We then proceed with some **classic NLP tasks, namely the removal of stopwords, tokenization and stemming.** The raw text is then transformed into a list of words, called tokens. **We turn the resulting words into numbers, more precisely arrays of numbers.** We will use three approaches: the first is very simple and based on word frequencies (TF-IDF), the others are more complex (Word2Vec, Doc2Vec). **The vectors resulting from these operations will be used by machine learning models** to make predictions about new comments. In particular **we will test different models**: logistic regression, naive bayes and linear svm. At the end we will choose **the combination of encoding and model that perform better**, and we will proceed with the **fine tuning, looking for the best parameters and the best threshold.** I also developed **a demo application** for this classifier, hosted on my Hugging Face. 

The following image shows graphically the workflow.

# ![title](../input/images/ToxicCommPipe.png)

## Steps of the Project
This is an end-to-end project covering all the necessary steps from text preprocessing to final classification and tuning.

The project consists of the following main sections (wich may consist of different subsections):

1. **Importing Tools**
2. **Importing Data and Cleaning**
3. **Preprocessing Text - NLP pipeline**
4. **Some Visualization**
5. **Splitting the data in training and testing sets**
6. **Different Embeddings: TF-IDF, Word2Vec, Doc2Vec**
7. **Dealing with Imbalanced Classes**
8. **Features Normalization**
9. **Algorithm Selection and Hyperparameter tuning**
10. **Performance of the model**
11. **ROC, AUC, Precision-Recall Curve**
12. **Threshold Tuning**
13. **Demo App & Further Steps**
14. **Drawing conclusions — Summary**

## Dataset



The dataset for this project is hosted on Kaggle and was part of a competition. According to the description on Kaggle, *The Conversation AI team, a research initiative founded by Jigsaw and Google (both a part of Alphabet) are working on tools to help improve online conversation. One area of focus is the study of negative online behaviors, like toxic comments (i.e. comments that are rude, disrespectful or otherwise likely to make someone leave a discussion)*

**The goal of this project is slightly different from the original one: here i will perform a binary classification of the comments** (instead of the multiclass multioutput defined in the original competition).

I will use only the train set provided by Kaggle and i will proceed with the usual splitting into train and test sets. **The dataset consists of 143346 non-toxic comments and 16225 toxic comments. A comment is considered toxic if belongs to at least one of the toxicity categories provided in the original dataset.** The original toxicity labels have been assigned in a manual way by humans, according to the description of the dataset available on Kaggle.

The dataset contains the raw comments, extracted as they were originally posted by users. **This comments have been extracted from Wikipedia talks pages.** For more details about the dataset, please refer to the competition page on Kaggle, at <https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge>.

## 1. Importing Tools

For this project are required the common data science libraries. In addition i use **imblearn** to deal with imbalanced datasets, **nltk** and **spacy** for basic NLP tasks, **gensim** for advanced text embedding and **wordcloud** to provide visualizations base on the words in the corpus.

In [ ]:
# UTILITY
from PIL import Image
from joblib import dump
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.preprocessing import MinMaxScaler, Normalizer
import warnings

# IMBALANCED LEARN
from imblearn.over_sampling import SMOTE

# NLP
import re
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk import SnowballStemmer
import spacy
from gensim.models.doc2vec import Doc2Vec, TaggedDocument, Word2Vec
from gensim.test.utils import get_tmpfile
from sklearn.feature_extraction.text import TfidfVectorizer

# MODELS
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

# EVALUATION
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score,\
     f1_score, roc_auc_score, roc_curve, precision_recall_curve

# VISUALIZATION
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# INIT
warnings.filterwarnings('ignore')
NUM_OF_THREADS = 10 # set > 1 to use multithreading support

## 2. Importing Data and Cleaning

The first step of this project is importing and **cleaning textual data**. For this purpose we start with some **basic regex-based text cleaning** operations. 

In particular:

1. remove numbers and words concatenated with numbers (IE h4ck3r)
2. remove emails and mentions (words with @)
3. remove hashtags (words with #)
4. remove URLs
5. replace the ASCII '�' symbol with '8'
6. remove multiple spaces

In [ ]:
# read data from csv file
data = pd.read_csv("../input/traindata/train.csv")
# drop na and reset index
data.dropna(inplace=True)
data.reset_index(inplace=True, drop=True)
# column name to work with
name = "comment_text"

In [ ]:
# function to clean the text performin some simple regex pattern matching
def apply_regex(corpus):
    corpus = corpus.apply(lambda x: re.sub("\S*\d\S*"," ", x))          # removes numbers and words concatenated with numbers (IE h4ck3r)
    corpus = corpus.apply(lambda x: re.sub("\S*@\S*\s?"," ", x))        # removes emails and mentions (words with @)
    corpus = corpus.apply(lambda x: re.sub("\S*#\S*\s?"," ", x))        # removes hashtags (words with #)
    corpus = corpus.apply(lambda x: re.sub(r'http\S+', ' ', x))         # removes URLs
    corpus = corpus.apply(lambda x: re.sub(r'[^a-zA-Z0-9 ]', ' ',x))    # keeps numbers and letters
    corpus = corpus.apply(lambda x: x.replace(u'\ufffd', '8'))          # replaces the ASCII '�' symbol with '8'
    corpus = corpus.apply(lambda x: re.sub(' +', ' ', x))               # removes multiple spaces
    return corpus

In [ ]:
# apply the function and clean the data
data[name] = apply_regex(data[name])

As we said, in this project we focus **on a binary classification task**, so we create a new **binary column called *label*.** The value of this attribute is 1 if the comments belongs to at least oone of the toxicity categories.

In [ ]:
# ASSIGN A BINARY LABEL -- 1 is toxic, 0 is not
data["label"] = np.where(data["toxic"]==1, 1, 
                np.where(data["severe_toxic"]==1, 1, 
                np.where(data["obscene"]==1, 1, 
                np.where(data["threat"]==1, 1,
                np.where(data["insult"]==1, 1,
                np.where(data["identity_hate"]==1, 1, 0))))))

After this simple operation, we check if we are in an imbalanced situation. **The imbalanced ratio is almost 9:1**. I will deal with this problem using **SMOTE to oversample the minority class in order to balance the classes** with a 1:1 ratio. The following instruction shows the actual samples for each class. The balancing process **will be performed after the encoding** of the text into vecotrs, due to the fact that SMOTE works on numerical data.

In [ ]:
# Count of samples for each class
Counter(data["label"])

## 3. Preprocessing Text - NLP pipeline

After the inital steps, we need to proceed with a **NLP pipeline**. The are some steps which are common for NLP problems like text classification, in particular we need to:

1. Remove stop words and tokenize the text
2. Stemming the tokens

**Tokens are the building blocks of Natural Language**. Tokenization is a way of **separating a piece of text into smaller units** called tokens. Here, tokens can be either words, characters, or subwords but in **this project we tokenize words.**

**Stop word removal** is one of the most commonly used preprocessing steps across different NLP applications. The idea is simply **removing the words that occur commonly across all the documents in the corpus.** Typically, articles and pronouns are generally classified as stop words. **These words have no significance** in some of the NLP tasks like information retrieval and classification.

**Stemming** is a natural language processing technique that **lowers inflection in words to their root forms**, hence aiding in the preprocessing of text, words, and documents for text normalization. According to Wikipedia, inflection is the process through which a word is modified to communicate many grammatical categories, including tense, case, voice, aspect, person, number, gender, and mood. Thus, although a word may exist in several inflected forms, having multiple inflected forms inside the same text adds redundancy to the NLP process. **Stemming is not as good as lemmatization, but it is faster**. Even using stemming we are able to get good results at the end.

In [ ]:
# TOKENIZE TEXT - we use the Spacy library stopwords
spacy_model = spacy.load("en_core_web_sm")
stop_words = spacy_model.Defaults.stop_words

In [ ]:
# TOKENIZE TEXT and STOP WORDS REMOVAL - execution (removes also the words shorter than 2 and longer than 15 chars)
def tokenize(doc):
    tokens_1 = word_tokenize(str(doc))
    return [word.lower() for word in tokens_1 if len(word) > 1 and len(word) < 15 and word not in stop_words and not word.isdigit()]
    
data["tokenized"] = data[name].apply(tokenize)

In [ ]:
# STEMMING
stemmer = SnowballStemmer(language="english")

def applyStemming(listOfTokens):
    return [stemmer.stem(token) for token in listOfTokens]

data['stemmed'] = data['tokenized'].apply(applyStemming)

**The final result** of the NLP preprocessing is shown in the next cell. **We can see the result of all the steps**, from raw text to stemmed text. For the next steps, **we will use the stemmed text.**

In [ ]:
data.sample(3)

## 4. Some Visualizations

In this section we go throught some visualizations that will help us to better understand the data we are working with. This visualizations are **word clouds of the toxic and non-toxic raw comments.** This visualization are obtained using the *wordcloud* library. Word Clouds are visual displays of text data for simple text analysis. **Word Clouds display the most prominent or frequent words in a body of text.** 

First we divide toxic and non-toxic comment based on the binary label.

In [ ]:
# TAKING TOXIC COMMENTS
threat_text = data[data["label"]==1]["comment_text"]
neg_text = pd.Series(threat_text).str.cat(sep=' ')

# TAKING NON-TOXIC COMMENTS
non_threat_text = data[data["label"]==0]["comment_text"]
pos_text = pd.Series(non_threat_text).str.cat(sep=' ')

At this point, we generate the wordclouds. The choice of the color palette is arbitrary, i tried to select colors that enfatize the toxicity of the words.

In [ ]:
# Generate toxic word cloud
wordcloud_neg = WordCloud(collocations=False, 
                            width=1500, 
                            height=600, 
                            max_font_size=150
                        ).generate(neg_text)
plt.figure(figsize=(15, 10))
plt.imshow(wordcloud_neg.recolor(colormap="GnBu"), interpolation='bilinear')
plt.axis("off")
plt.title(f"Most common words assosiated with toxic comment", size=20)
plt.show()

In [ ]:
#Generate non-toxic word cloud
wordcloud_pos = WordCloud(collocations=False, 
                            width=1500, 
                            height=600, 
                            max_font_size=150
                        ).generate(pos_text)
plt.figure(figsize=(15, 10))
plt.imshow(wordcloud_pos.recolor(colormap="Accent"), interpolation='bilinear')
plt.axis("off")
plt.title(f"Most common words assosiated with non-toxic comment", size=20)
plt.show()

As we can see from this visualizations, while some words are present in both the categories, other are **really caracterizing the toxic comments.**

## 5. Splitting the Data in Training and Testing sets

The next step consist of **splitting the dataset into train and test sets** using the usual *train_test_split* functionality provided by sklearn.

In [ ]:
# Even if for the competition on kaggle the gave also the "test" dataset, i decided to use only the "train" set as a unique dataset.
X_train, X_test, y_train, y_test = train_test_split(data["stemmed"], data["label"], random_state=42)

## 6. Different Embeddings: TF-IDF, Word2Vec, Doc2Vec

Now that we have the test and train sets, we need to **encode the stemmed text**, that now is a vector of words, **into a Vector Space Model (VSM)** that means **converting the array of words into an array of numbers**, that will be used by a machine learning algorithm for the actual classification task.

Converting text into a VSM is a **non trivial aspect, involving some domain-based reasoning.** In this project i will encode the text using **three different methods**:

1. Basic TF-IDF (frequency based) approach
2. More advanced Word2Vec model (custom trained on the corpus)
3. More advanced Doc2Vec model (custom trained on the corpus)

This approaches will be explained in the following subsections.

### TF-IDF Vectorizer

According to Wikipedia, *TF-IDF, short for **term frequency–inverse document frequency**, is a numerical **statistic that is intended to reflect how important a word is to a document in a collection or corpus.** It is often used as a weighting factor in searches of information retrieval, text mining, and user modeling. The tf–idf value increases proportionally to the number of times a word appears in the document and is offset by the number of documents in the corpus that contain the word, which helps to adjust for the fact that some words appear more frequently in general. **tf–idf is one of the most popular term-weighting schemes today.** A survey conducted in 2015 showed that 83% of text-based recommender systems in digital libraries use tf–idf. Variations of the tf–idf weighting scheme are often used by search engines as a central tool in scoring and ranking a document's relevance given a user query. tf–idf can be successfully used for stop-words filtering in various subject fields, including text summarization and classification.*

In [ ]:
# commodity methos to avoid running tokenization again
def do_nothing(lemmas):
    return lemmas

The stemmed text is passed to the sklearn *TfidfVectorizer* specifying a number of features equal to 500. This is an arbitrary choice, and in order to have a fair comparison with other method, this parameter will be setted to 500 for all the three methods.

In [ ]:
# encodin text into vectors
tfid = TfidfVectorizer(lowercase=False, tokenizer=do_nothing, max_features=500)

train_vectors_tfidf = tfid.fit_transform(X_train).toarray()
test_vectors_tfidf = tfid.transform(X_test).toarray()

### Word2Vec: Model Creation and Persistence Store

Word2vec is a technique for natural language processing published in 2013. This algorithm uses a **neural network to learn word associations** from a large corpus of text. Once trained, such a model can detect synonymous words or suggest additional words for a partial sentence. As the name implies, **word2vec represents each distinct word with a particular list of numbers called a vector.** The vectors are chosen carefully such that a simple mathematical function (the cosine similarity between the vectors) indicates the level of semantic similarity between the words represented by those vectors.

Word2Vec represent each word as a vector, but in this project we are working with comments, that are a set of words. In order to obtain a representation of a document (a comment) we **take the average of the representations of the words that make that document (comment).** This is a very simple but still effective approach.

We use Word2Vec in order to create a **continuous bag-of-words (CBOW) representation**. In the continuous bag-of-words architecture, **the model predicts the current word from a window of surrounding context words**. The order of context words does not influence prediction, and this could be a limitation since the meaning could be lost. Two sentences like "men eats dog" and "dog eats men" will be encoded in the same way, but the meaning is completely different.

We start by setting the parameter for the train of the model. Once ther model is trained, we save it on the disk.

In [ ]:
# execute only if in order to retrain the model or if the trained model is not present in the folder "./w2vec.model"
# seting the parameter for the w2v model
w2v = Word2Vec(
                sg=0,                   # 1 for skip-gram, 0 for CBOW
                vector_size=500,        # size of the resulting vector
                workers=NUM_OF_THREADS, # multithreading
                epochs=100,             # number of epochs
                seed=42                 # seed for reproducibility
            )            

# building the vocabulary and training the model
w2v.build_vocab(X_train, progress_per=50000)
w2v.train(X_train, total_examples=w2v.corpus_count, epochs=30, report_delay=1)

# saving the model to the disk in order to avoid training again
w2v.save("w2vec.model")
print("Model Saved")

### Word2Vec Embedding using the Trained Model

At this point, we can load the trained model and use it for the actual embedding.

In [ ]:
wordVec = Word2Vec.load("./w2vec.model").wv

The following function takes the tokens of each document (comment) and look up in the model for the vectorial representation of each token. All the representations of the tokens of each document are averaged together in order to get a vectorial representation of the whole comment. **Basically a comment is represented as the average of the words representations of the words that make that comment.**

In [ ]:
# Creating a feature vector by averaging all embeddings for all sentences
def embedding_feats(corpus, w2v_model):
    DIMENSION = 500
    zero_vector = np.zeros(DIMENSION)
    feats = []
    for tokens in corpus:
        feat_for_this =  np.zeros(DIMENSION)
        count_for_this = 0 + 1e-5 # to avoid divide-by-zero 
        for token in tokens:
            if token in w2v_model:
                feat_for_this += w2v_model[token]
                count_for_this +=1
        if(count_for_this!=0):
            feats.append(feat_for_this/count_for_this) 
        else:
            feats.append(zero_vector)
    return feats


train_vectors_w2v = embedding_feats(X_train, wordVec)
test_vectors_w2v = embedding_feats(X_test, wordVec)

The resulting vectors will be used in the next steps.

### Doc2Vec: Model Creation and Persistence Store

**Doc2Vec model**, as opposite to Word2Vec model, is used to **create a vectorised representation of a group of words taken** collectively as a single unit. It doesn’t only give the simple average of the words in the sentence. With this approach we directly represent a document (comment) into a vector.

The **Distributed Bag-Of-Words (DBOW)** model helps in guessing the context words from a target word. The difference between the distributed bag of words and the **Distributed Memory Model** is that the Distributed Memory Model approximates the word using the context of surrounding words and the distributed bag of words model uses the target word to approximate the context of the word.

We prepare the data to be trained and set up the parameters. Once the model is trained, we save it on the disk.

In [ ]:
# execute only if in order to retrain the model or if the trained model is not present in the folder "./d2v_comments.model"
#prepare training data in doc2vec format:
train_doc2vec = [TaggedDocument((d), tags=[str(i)]) for i, d in enumerate(X_train)]

#Train a doc2vec model to learn representations.
model = Doc2Vec(
                vector_size=500,                # size of the resulting vector
                dm=0,                           #dm=0 -> we use a DBOW representation
                epochs=100,                     # number of epochs
                seed=42,                        # seed for reproducibility
                workers=NUM_OF_THREADS,         # multithreading
                )        

# building the vocabulary and training the model
model.build_vocab(train_doc2vec)
model.train(train_doc2vec, total_examples=model.corpus_count, epochs=model.epochs)

# saving the model to the disk in order to avoid training again
model.save("d2v_comments.model")
print("Model Saved")

### Doc2Vec Embedding using the Trained Model

We load the trained model and with the *infer_vector* method we generate a vectorized representation of each document. This instruction will take some time.

In [ ]:
#Infer the feature representation for training and test data using the trained model
model_d2v= Doc2Vec.load("d2v_comments.model")

#infer in multiple steps to get a stable representation. 
train_vectors_d2v =  [model_d2v.infer_vector(tokens, epochs=70) for tokens in X_train]
test_vectors_d2v = [model_d2v.infer_vector(tokens, epochs=70) for tokens in X_test]

At the end of this section we have **three different encodings obtained with different methods**. The next section is about dealing with imbalanced classes.

## 7. Dealing with Imbalanced Classes

The dataset is **imbalanced with a ratio of almost 9:1.** In order to deal with this problem we use the *imbalanced learn* libraries. In particular i'm using the **SMOTE (synthetic minority oversampling technique) algorithm to produce new synthetic data points**. This new samples are generated using vector operations (this is why this operation must be done after the vector encoding phase). With the default parameters, the algorithm will **completely balance the dataset**, that will end up having a ratio of 1:1, that means same number of observation for both the classes.

In particular, SMOTE selects a minority class instance (a) at random and finds its k nearest minority class neighbors. Then, one of the k nearest neighbors is choosen at random (b). Using any distance metric, the difference in distance between the feature vector (a) and its neighbor (b) is computed. Now, this difference is multiplied by any random value in (0,1] and is added to the previous feature vector (a) in order to obtain the synthetic sample. 

The operation is applied to train and test data resulting from each of the encoding methods used, in order to keep different approaches separated. I decided to use SMOTE because according to *"V. Rupapara et al.: Impact of SMOTE on Imbalanced Text Features for Toxic Comments Classification"* this approach is very effective in this kind of applications. After some tests i realized that this operations really seems to **boost the performance of the final classification model** for this project.

In [ ]:
# SMOTE for the tf-idf encoding
sm_tfidf = SMOTE(random_state=42, n_jobs=NUM_OF_THREADS)
train_vectors_tfidf, y_train_tfidf = sm_tfidf.fit_resample(train_vectors_tfidf, y_train)
test_vectors_tfidf, y_test_tfidf = sm_tfidf.fit_resample(test_vectors_tfidf, y_test)

In [ ]:
# SMOTE for the W2V encoding
sm_w2v = SMOTE(random_state=42, n_jobs=NUM_OF_THREADS)
train_vectors_w2v, y_train_w2v = sm_w2v.fit_resample(train_vectors_w2v, y_train)
test_vectors_w2v, y_test_w2v = sm_w2v.fit_resample(test_vectors_w2v, y_test)

In [ ]:
# SMOTE for the D2V encoding
sm_d2v = SMOTE(random_state=42, n_jobs=NUM_OF_THREADS)
train_vectors_d2v, y_train_d2v = sm_d2v.fit_resample(train_vectors_d2v, y_train)
test_vectors_d2v, y_test_d2v = sm_d2v.fit_resample(test_vectors_d2v, y_test)

## 8. Features Normalization

In this section **we normalize the data** using *normalizer* from sklearn. *normalizer* **normalizes samples individually to unit norm.** Each sample with at least one non zero component is rescaled independently of other samples so that its norm (l1, l2 or inf) equals one.
Scaling inputs to unit norms is a **common operation for text classification** or clustering for instance. We execute a normalization for each method used to encode the text. **The resulting normalized vectors will be fed as input to the classification models.** After some tests, this approach seems to improve the final performance of the model.

We normalize the vectors obtained from the different encoding techniques.

### TF-IDF Normalization

In [ ]:
# NORMALIZING TF-IDF VECTORS
norm_TFIDF = Normalizer(copy=False)
norm_train_tfidf = norm_TFIDF.fit_transform(train_vectors_tfidf)
norm_test_tfidf = norm_TFIDF.transform(test_vectors_tfidf)

### Word2Vec Normalization

In [ ]:
# NORMALIZING WORD2VEC VECTORS
norm_W2V = Normalizer(copy=False)
norm_train_w2v = norm_W2V.fit_transform(train_vectors_w2v)
norm_test_w2v = norm_W2V.transform(test_vectors_w2v)

### Doc2Vec Normalization

In [ ]:
# NORMALIZING DOC2VEC VECTORS
norm_D2V = Normalizer(copy=False)
norm_train_d2v = norm_D2V.fit_transform(train_vectors_d2v)
norm_test_d2v = norm_D2V.transform(test_vectors_d2v)

At the end of this section, **we have a train and a test set for each encoding method used.** This vectors will be used for the classification task.

## 9. Algorithm Selection & Hyperparameter Tuning

In this section **different baseline models will be tested** for each encoding methods used.
Basically for each encoding techniques (TF-IDF, Word2VEc, Doc2Vec) will be tested 3 models (for a total of 9 models):

1. Linear SVC baseline
2. Logistic Regression baseline
3. Naive Bayes baseline

A custom function will be used to **reason on the performance of these models** and, at the end, one model will be selected. The selected combination of model and encoding will be fine tuned.

We create a custom function to test programmatically different model baselines.

In [ ]:
# generic function to create different models
def create_models(seed=42, classW="balanced"):
    models = []
    # we can append more than one model to test
    models.append(('LinearSVC', LinearSVC(random_state=seed, class_weight=classW)))
    models.append(('Logit', LogisticRegression(random_state=seed, class_weight=classW, n_jobs=NUM_OF_THREADS, max_iter=300)))
    models.append(('NaiveBayesMN', MultinomialNB()))
    return models

In order to evaluate performance of different techniques and models, a custom function is used. The following function **return a dataframe with some evaluation metrics**. <br><br>
The metrics considered are:
* Accuracy
* F1 score
* Recall
* Precision
* Specificity
* True Positive num
* True Negative num
* False Positive num
* False Negative num
* Size of the Test Set

In [ ]:
# Returns a dataframe with classification metrics and confusion matrix values
def make_classification_score(y_test, predictions, modelName):
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    prec=precision_score(y_test, predictions)
    rec=recall_score(y_test, predictions)
    f1=f1_score(y_test, predictions)
    acc=accuracy_score(y_test, predictions)
    # specificity
    spec=tn/(tn+fp)

    score = {'Model': [modelName], 'Accuracy': [acc], 'f1': [f1], 'Recall': [rec], 'Precision': [prec], \
        'Specificity': [spec], 'TP': [tp], 'TN': [tn], 'FP': [fp], 'FN': [fn], 'y_test size': [len(y_test)]}
    df_score = pd.DataFrame(data=score)
    return df_score

Another custom function is used in order to **automatically train - test - evaluate** all the models defined.

In [ ]:
# generic function to test the created models
def test_models(models, X_train, y_train, X_test, y_test):
    score = pd.DataFrame()
    for name, model in models:
        # in case of nayve bayes we scale the values
        if name=="NaiveBayesMN":
            scaler = MinMaxScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
        # fit the model with the training data
        model.fit(X_train, y_train)
        # make predictions with the testing data
        preds = model.predict(X_test)
        # compute score
        tmp = make_classification_score(y_test, preds, name)
        score = pd.concat([score, tmp])
    return score

We proceed with the actual **creation of the models**, after this step, different encoding approaches are tested in the following sub sections.

In [ ]:
models = create_models()

### Testing the TF-IDF Approach

In [ ]:
results = test_models(models, norm_train_tfidf, y_train_tfidf, norm_test_tfidf, y_test_tfidf)
results

### Testing the Word2Vec Approach

In [ ]:
results = test_models(models, norm_train_w2v, y_train_w2v, norm_test_w2v, y_test_w2v)
results

### Testing the Doc2Vec Approach

In [ ]:
results = test_models(models, norm_train_d2v, y_train_d2v, norm_test_d2v, y_test_d2v)
results

### RandomizedSearch to find the Best Parameters

As result of the previous section, we identified a good encoding and a good model: **Logistic Regression with Doc2Vec seems better than the other candidates**. Now we perform an **hyperparameter search** in order to **find the best parameters**. For this purpose we **set up a parameter grid for the model**. The complete list of parameter for each model can be found on the documentation provided by sklearn.

In [ ]:
#### LOGIT PARAM GRID ####
logit_param = {"penalty" : ["l1","l2","elasticnet"],
                "tol" : [0.0004, 0.004, 0.04, 0.4],
                "C" : [0.5, 1, 1.5, 2, 5],
                "random_state" : [42],
                "solver" : ["saga", "sag", "liblinear", "lbfgs", "newton-cg"],
                "max_iter" : [200,400,600,800]
            }

We run the **randomized search, with a cross validation folds set to 8 and 15 iterations.** At the end we can get also the best parameters found by the search. Here another option is *GridSearchCV* that tests all the possible combinations of parameters. Even if a grid search represents a more exhaustive search, it turns out to be computationally too complex and long compared to a randomized search. Basically, in contrast to GridSearchCV, not all parameter values are tried out, but rather a fixed number of parameter settings is sampled from the specified distributions. **The number of parameter settings that are tried is given by n_iter.** 

The random search approach could lead to a non-ptimal combination of parameter (a sort of local optimum), so **we are not sure that we have found the real best parameters**.

In [ ]:
# running the search and fitting the model
final_model = RandomizedSearchCV(LogisticRegression(), 
                                logit_param, 
                                random_state=42, 
                                cv=8, 
                                verbose=-1, 
                                n_jobs=NUM_OF_THREADS, 
                                n_iter=15)
                                
final_model.fit(norm_train_d2v, y_train_d2v)
# make predictions with the testing data
preds = final_model.predict(norm_test_d2v)
# compute score
make_classification_score(y_test_d2v, preds, "logit_tuned_d2v")

In [ ]:
# return the possibly best estimator
final_model.best_estimator_

The previous step returned the best parameters for the algorithms. Now we can use the *predict_proba()* method to **predict the probability of belonging to one class or another.** We will use this probabilities in other to tune a custom threshold.

In [ ]:
# getting predicted probability values
probability = final_model.predict_proba(norm_test_d2v)

We conclude this section by **materializing the fitted model on the disc**, in order to use it in other application or avoid retraining.

In [ ]:
dump(final_model, 'toxicCommModel.joblib')

## 10. Performance of the Model

Evaluating the performance of the model is a very important step. In order to do so, i created two custom functions. The first to **convert the probability of belonging to a class into a final binary class** using a specified threshold. The second one return a dataframe with all the **metrics for the evaluation of the model**. This functions are described in the following sections. We use the probability from the previous step to assign to each sample a final class.

### Custom Function to convert a probability to a class using a threshold t

This function takes the probability from *predict_proba()* and a threshold value. **Then convert the probability to a binary class [0,1]**. Note that in the function we **consider the prbability to belong to the "1" class (toxic comment).**

In [ ]:
# from probability to class value based on custom threshold
def probs_to_prediction(probs, threshold):
    pred=[]
    for x in probs[:,1]:
        if x>threshold:
            pred.append(1)
        else:
            pred.append(0)
    return pred

### Model Evaluation with a Threshold = 0.5
Before doing some advanced threshold tuning, it is important to **check that results with a threshold t=0.5 are the same obtained previously** using the *predict()* method. **Results are consistent because t=0.5 is the default threshold implementated by the algorithm.**

In [ ]:
predictions=probs_to_prediction(probability, 0.5)
make_classification_score(y_test_d2v, predictions, "logit, t=0.5")

The result is not bad and all the considered metrics are over .91. **We still have some False Positives and False Negatives**: this leads us to a **more advanced threshold tuning** in order to decide, based on the application, **which is the important metric to maximize** (precision or recall based on the cost of a FN and the cost of a FP)

## 11. ROC, AUC, Precision-Recall Curve

Before tuning the threshold of our classifier, in this section we plot different curves. This plots will help to decide the **"trade off" between all the metrics**. The threshold choice starts from this trade off.

### ROC AUC and Curve Plot

By definition an ROC curve (receiver operating characteristic curve) is a graph showing the **performance of a classification model at all the classification thresholds**. First we compute the Area Under The roc Curve (AUC) and then we plot the curve. **A value of the AUC=1 refers to a perfect classifier, AUC=0.5 refers to a random classifier.**

In [ ]:
# calculate roc auc score
print("SVM: ROC AUC = %.4f" % roc_auc_score(y_test_d2v, probability[:, 1]))

We got an **AUC value equal to 0.9794**, not bad. Now we **plot the curve**. The dots represent **different threshold values.**

In [ ]:
# calculate roc curve
model_fpr, model_tpr, _ = roc_curve(y_test_d2v, probability[:, 1])
# plot roc curve
plt.plot(model_fpr, model_tpr, marker='.', label='Logit')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (recall)')
plt.legend()
plt.title("ROC Curve")
plt.show()

### Precision-Recall Cruve Plot

In a situation of unbalanced classes it is better to use a Precision - Recall curve instead of a ROC curve. Even if we balanced the dataset, caould be usefull to look at this curve anyway.<br>
The precision-recall curve shows the **tradeoff between precision and recall** for different threshold. **We can choose a value of the threshold that matches to de desired values of precision and recall.**

In [ ]:
# calculate precision-recall curve
model_precision, model_recall, thresholds = precision_recall_curve(y_test_d2v, probability[:, 1])
# plot the curve
plt.plot(model_recall, model_precision, marker='.', label='Logit')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.title('Precision-Recall Curve')
plt.show()

## 12. Threshold Tuning

**The choice of the important metric** to maximize (precision vs recall) **depends on the domain and the application itself**. For example, in spam detection if an important email were marked as spam when it actually isn’t, this could be disastrous as you would miss it. As opposite, in medicine a false negative would be much more damaging.

**Based on the context** in which we use a toxic comments classification model, **we can decide which metric to maximize based on the cost of a false positive or false negative**, there is no a right or wrong answer here. In an application for kids (YouTube Kids) we would like to maximize the recall as a False Negative detains an higher misclassification cost. In another context we could maximize precision in order to avoid the reporting of non-toxic comments.

**Let's try, for instance, to maximize precision that means cost of FP > cost of FN.** This operation requires some tests at different thresholds until we are satisfied with the number of FN and FN.

In [ ]:
# plotting precision-recall curve with the choosen threshold
plt.plot(model_recall, model_precision, marker='.', label='Logit')
plt.plot(model_recall[43000], model_precision[43000], "ro", label="threshold")
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.title('Precision and Recall values for choosen Threshold')
plt.show()

The value of the choosen threshold is:

In [ ]:
# print the threshold value
print("Threshold value = %.4f" % thresholds[43000])

As we see, the **number of false positive with this threshold is 266 and the precision is 0.9898**. On the other hand, the recall dropped to 0.730.

In [ ]:
# choosen threshold results
predictions=probs_to_prediction(probability, thresholds[43000])
make_classification_score(y_test_d2v, predictions, "logit, custom t")

We can try also different solutions. **It is up to us to decide what is better given the actual costs** of misclassification.

## 13. Demo App & Further Steps

With the pipeline and the trained models, is possible to **develop a demo application with a GUI**. These models can also be **embedded in existing applications**.

For this purpose, i developed a demo application to play a bit with the tuned models. The application is hosted at <https://huggingface.co/spaces/EdBianchi/Social_Toximeter>.

The demo consist of **three sections**, the **first one allow to write a comment and display the probabilities** assigned to each class by the classifier.

<img src="../input/images/app1.png" width="550" height="350">

**The second one shows the final classification label for the comment at three different thresholds**, 0.5 is the default thresh, 0.937 is the thresh selected after the tuning and 0.999 is a more extreme thresh. **This allow to understand how the output class changes based on the threshold** (and the importance of choiching a correct threshold).

<img src="../input/images/app2.png" width="550" height="200">

**The last section shows the output of the NLP preprocerssing pipeline**, this are the tokens that are embedded using the trained D2V model.

<img src="../input/images/app3.png" width="500" height="350">

## 14. Drawing Conclusions - Summary

We started by **cleaning the data** and performing a common **NLP pipeline**. After, we used **different embedding methods**, from a frequency based one to more advanced ones. With the resulting vectors **we performed class balancing and normalization**. At the end we **tested different baseline** models and **tuned the hyperparameters** of the best performing model. At the end we selected a context in which precision is the important metric to maximize, and **we tuned the threshold**.

I am satisfied with this result of this work, and surprised about the effect of SMOTE on the performance of the model. I think that working on natural language, as well as being the basis of important everyday applications, is a stimulating and intriguing challenge: the whole society in which we live is based on language (which allows us to communicate).

I tried to follow and cover **all the main steps involved in a real Data Science pipeline** applied to a NLP project and be **as precise as possible in the description** of all the passages. Although the notebook shows a linear process, **machine learning projects tend to be iterative rather than linear processes**, where previous steps are often revisited as we learn more about the problem to solve. I did lots of experiments with the encoding techniques, with normalization and class balancing in order to find this solution.

I conclude keeping in my mind that *"All models are wrong, but some are useful"*, i hope this can be a useful one.

Bianchi Edoardo